In [ ]:
import kagglehub
import os

folder = 'new-york-city-transport-statistics'

# Baixa os arquivos de GPS
path = kagglehub.dataset_download(os.path.join('stoney71', folder))
print('opath:', path)


In [ ]:
## usar no colab
# dpath = '/content/data_raw'

# usar no vscode
dpath = os.path.abspath('../data/raw')

dpath

In [ ]:
# Cria a para que conterá os arquivos originais baixados
os.system(f'mkdir -p {dpath}')

# Caminhos de origem do arquivo baixado do Kaggle e Destino como pasta local para
opath = os.path.join(path)
dpath = os.path.join(dpath)

print('opath:', opath)
print('dpath:', dpath)

# Copia todos os arquivos da pasta do Kaggle para data_raw
os.system(f'mv {opath}/* {dpath}/')
os.system(f'rm -rf {opath}')

In [ ]:
# # Usar quando estiver no colab
# from google.colab import auth
# auth.authenticate_user()

# # Usar quanto estiver no vscode
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../gcp_key.json"

In [ ]:
from google.cloud import storage

# Cria o cliente para acessar o bucket
client = storage.Client()

# Referência pro bucket do GCP
bucket = client.bucket('etl_bus_gps')

In [ ]:
# Faz o upload de todos os arquivos para o bucket
for filename in os.listdir(dpath):
    file_path = os.path.join(dpath, filename)

    blob = bucket.blob(f'raw/{filename}')
    blob.upload_from_filename(file_path)
    print(f'Arquivo "{filename}" enviado para a pasta "raw" no bucket {bucket.name}')